In [ ]:
COLAB = False

### Path setup

In [ ]:
# noqa
import os
if COLAB:
    %cd /content
    ROOT_DIR = '/content'
    REPO_DIR = os.path.join(ROOT_DIR, 'ml_project_template')
    LOG_DIR = os.path.join(REPO_DIR, 'data_out')

    if not os.path.isdir(REPO_DIR):
      !git clone https://github.com/oskopek/ml_project_template.git
    if not os.path.isdir(LOG_DIR):
      os.makedirs(LOG_DIR)
    %cd 'ml_project_template'
    %ls
else:
    wd = %pwd
    print('Current directory:', wd)
    if wd.endswith('notebooks'):
        %cd ..

### Install missing packages

Do not want to do `pip install -r requirements.txt` because that will overwrite the versions on Colab

In [ ]:
# noqa
if COLAB:
    !pip install dotmap==1.2.20

### Branch selection

In [ ]:
# noqa
if COLAB:
    !git checkout master

### Tensorboard setup

In [ ]:
# noqa
if COLAB:
    import os
    import resources.colab_utils.tboard as tboard

    # will install `ngrok`, if necessary
    # will create `log_dir` if path does not exist
    tboard.launch_tensorboard(bin_dir=REPO_DIR, log_dir=LOG_DIR)

## Actual notebook

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from models.base import KerasModel

# Flags
from flags import flags_parser
flags_parser.parse('flags/cnn.json', None)
FLAGS = flags_parser.FLAGS
assert FLAGS is not None

In [ ]:
# Model
class MnistCnn(KerasModel):
    # Setup constants
    IMAGE_SIZE = 28

    def __init__(self):
        super(MnistCnn, self).__init__(
            logdir=FLAGS.data.out_dir, expname="MNIST-CNN", threads=FLAGS.training.threads, seed=FLAGS.training.seed)

    # Construct the graph
    def _build(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=self.input_shape))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(FLAGS.model.arch.num_classes, activation='softmax'))

        optimizer = keras.optimizers.Adadelta(lr=FLAGS.model.optimization.learning_rate)
        model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

        self.model = model

    def _load_data(self):
        # the data, shuffled and split between train and test sets
        (x_train, y_train), (x_test, y_test) = mnist.load_data()

        if K.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1, self.IMAGE_SIZE, self.IMAGE_SIZE)
            x_test = x_test.reshape(x_test.shape[0], 1, self.IMAGE_SIZE, self.IMAGE_SIZE)
            self.input_shape = (1, self.IMAGE_SIZE, self.IMAGE_SIZE)
        else:
            x_train = x_train.reshape(x_train.shape[0], self.IMAGE_SIZE, self.IMAGE_SIZE, 1)
            x_test = x_test.reshape(x_test.shape[0], self.IMAGE_SIZE, self.IMAGE_SIZE, 1)
            self.input_shape = (self.IMAGE_SIZE, self.IMAGE_SIZE, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255
        print('x_train shape:', x_train.shape)
        print(x_train.shape[0], 'train samples')
        print(x_test.shape[0], 'test samples')

        # convert class vectors to binary class matrices
        y_train = keras.utils.to_categorical(y_train, FLAGS.model.arch.num_classes)
        y_test = keras.utils.to_categorical(y_test, FLAGS.model.arch.num_classes)
        return (x_train, y_train), (x_test, y_test)

    def run(self):
        (x_train, y_train), (x_test, y_test) = self._load_data()
        self._build()

        self.model.fit(
            x_train,
            y_train,
            batch_size=FLAGS.model.optimization.batch_size,
            epochs=FLAGS.model.optimization.epochs,
            verbose=1,
            validation_data=(x_test, y_test),
            callbacks=self.callbacks)

        score = self.model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

In [ ]:
# Run
MnistCnn().run()